<a href="https://colab.research.google.com/github/Doeschate/Research/blob/main/Datasets_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install NAM from github AND <font color='red'>**RESTART colab after install**</font>
# https://stackoverflow.com/questions/13566200/how-can-i-install-from-a-git-subdirectory-with-pip
!pip3 install -U --verbose -e "git+https://github.com/agarwl/google-research.git#egg=neural_additive_models&subdirectory=neural_additive_models" # install a python package from a repo subdirectory

In [ ]:
#@title Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import random
random.seed(365)

import data_utils
import models as nam_models
import graph_builder
import os.path as osp
import numpy as np


import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
#@title Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#@title Dataset helpers

def load_col_min_max(dataset_name):
  """Loads the dataset according to the `dataset_name` passed."""
  if dataset_name == 'Housing': ##
    dataset = data_utils.load_california_housing_data()
  elif dataset_name == 'BreastCancer':
    dataset = data_utils.load_breast_data()
  elif dataset_name == 'Recidivism':
    dataset = data_utils.load_recidivism_data()
  elif dataset_name == 'Fico': ##
    dataset = data_utils.load_fico_score_data()
  elif dataset_name == 'Mimic2': ##
    dataset = load_mimic2_data()
  elif dataset_name == 'Credit':
    dataset = data_utils.load_credit_data()
  else:
    raise ValueError('{} not found!'.format(dataset_name))

  if 'full' in dataset:
    dataset = dataset['full']
  x = dataset['X']
  col_min_max = {}
  for col in x:
    unique_vals = x[col].unique()
    col_min_max[col] = (np.min(unique_vals), np.max(unique_vals))
  return col_min_max

def inverse_min_max_scaler(x, min_val, max_val):
  return (x + 1)/2 * (max_val - min_val) + min_val 

In [ ]:
#@title Load dataset
#BreastCancer/Recidivism/Credit
dataset_name = 'BreastCancer' #@param {'type': 'string'}
is_regression = dataset_name in ['Housing', 'Fico']
data =  data_utils.load_dataset(dataset_name)
data_x, data_y, column_names = data
col_min_max = load_col_min_max(dataset_name)

In [ ]:
len(data_y)

569

In [ ]:
#@title Create train/test split

(x_train_all, y_train_all), (x_test_all, y_test_all) = data_utils.get_train_test_fold(
      data_x, data_y, fold_num=1, num_folds=5, stratified=not is_regression)


# data_gen = data_utils.split_training_dataset(
#       x_train_all, y_train_all, 
#       n_splits=20, stratified=not is_regression)


# (x_train, y_train), _ = next(data_gen)

In [ ]:
print(len(y_test_all))
print(len(y_train_all))

114
455


In [ ]:
#@title Custom Dataset
class CustomDataset:
  def __init__(self,data,targets):
    self.data = data
    self.targets = targets
  def __len__(self):
    return self.data.shape[0]
  def __getitem__(self,idx):
    current_sample = self.data[idx,:]
    current_target = self.targets[idx]
    return {
        "x": torch.tensor(current_sample,dtype=torch.float),
        "y": torch.tensor(current_target,dtype=torch.long),
    }

In [ ]:
train_dataset = CustomDataset(data=x_train_all, targets= y_train_all)
test_dataset = CustomDataset(data=x_test_all, targets= y_test_all)

In [ ]:
print(train_dataset[5]["x"].shape[0])
print(train_dataset[5]["y"])

30
tensor(0)


In [ ]:
#@title Parameters
#LinearModel/nlModel/dnnModel
model_name = 'LinearModel' #@param {'type': 'string'}
batch_size = 4
num_epochs = 10
learning_rate = 0.001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2

In [ ]:
class LinearModel(nn.Module):
  def __init__(self,input_size,output_size):
    super(LinearModel, self).__init__()
    print("LinearModel")
    self.ll1 = nn.Linear(input_size,10)
    self.ll2 = nn.Linear(10,output_size)
  def forward(self,inputs):
    x = self.ll1(inputs)
    x = self.ll2(x)
    return x

In [ ]:
#@title NTK
#install JAX on CPU by running
!pip install jax jaxlib --upgrade
#Once JAX is installed install Neural Tangents by running
!pip install neural-tangents

In [ ]:
class nlModel(nn.Module):
  def __init__(self,input_size,output_size):
    super(nlModel, self).__init__()
    print("nlModel")
    self.nl1 = nn.Linear(input_size,10)
    self.nl2 = nn.Linear(10,output_size)

  def forward(self,inputs):
    x = F.relu(self.nl1(inputs))
    x = self.nl2(x)

    return x

In [ ]:
class dnnModel(nn.Module):
  def __init__(self,input_size,output_size):
    super(dnnModel, self).__init__()
    print("dnnModel")
    self.nl1 = nn.Linear(input_size,10)
    self.nl2 = nn.Linear(10,10)
    self.nl3 = nn.Linear(10,10)
    self.nl4 = nn.Linear(10,output_size)

  def forward(self,inputs):
    x = F.relu(self.nl1(inputs))
    x = F.relu(self.nl2(x))
    x = F.relu(self.nl3(x))
    x = self.nl4(x)

    return x

In [ ]:
if model_name == 'nlModel':
  model = nlModel(input_size,output_size).to(device)
if model_name == 'LinearModel':
  model = LinearModel(input_size,output_size).to(device) 
if model_name == 'dnnModel':
  model = dnnModel(input_size,output_size).to(device) 


LinearModel


In [ ]:
criterion = nn.CrossEntropyLoss()
if optimizer_name == 'SGD':
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
if optimizer_name == 'Adam':
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#@title Train Dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False)


In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  i=0
  for train_data in train_loader:
    i=i+1
    xtrain=train_data["x"]
    ytrain=train_data["y"]
    xtrain = xtrain.to(device)
    ytrain = ytrain.to(device)
   
    # Forward pass
    outputs = model(xtrain)
    loss = criterion(outputs, ytrain)
    torch.cuda.empty_cache()
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/10], Step [100/114], Loss: 0.5546
Epoch [2/10], Step [100/114], Loss: 0.6843
Epoch [3/10], Step [100/114], Loss: 0.4200
Epoch [4/10], Step [100/114], Loss: 0.3658
Epoch [5/10], Step [100/114], Loss: 0.6214
Epoch [6/10], Step [100/114], Loss: 0.6052
Epoch [7/10], Step [100/114], Loss: 0.6496
Epoch [8/10], Step [100/114], Loss: 0.4376
Epoch [9/10], Step [100/114], Loss: 0.5316
Epoch [10/10], Step [100/114], Loss: 0.2766
Finished Training


In [ ]:
outputs = []
labels = []
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for test_data in test_loader:
    xtest=test_data["x"]
    ytest=test_data["y"]
    xtest = xtest.to(device)
    ytest = ytest.to(device)

    output = model(xtest)
    labels.append(ytest)

    # max returns (value ,index)
    _, predicted = torch.max(output, 1)
    outputs.append(predicted)

    n_samples += ytest.size(0)
    n_correct += (predicted == ytest).sum().item()
  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

  

Accuracy of the network: 67.54385964912281 %


In [ ]:
# import jax.numpy as np
# import neural_tangents as nt
# from neural_tangents import stax
# init_fn, apply_fn, kernel_fn = stax.serial(
#     stax.Dense(10), stax.Relu(),
#     stax.Dense(10), stax.Relu(),
#     stax.Dense(1)
# )

# x_train = xtrain
# x_test = xtest
# y_train = ytrain
# y_test = ytest
# kernel = kernel_fn(x_train, x_test, 'ntk')
 

# ntk_train_train = kernel_fn(x_train, None, 'ntk')

# ntk_test_train = kernel_fn(x_test, x_train, 'ntk')

# mse_predictor = nt.predict.gradient_descent_mse(ntk_train_train, y_train)

# # t = 5.
# y_test_ntk = mse_predictor(x_test=x_test, get='ntk')
# # y_train_t, y_test_t = mse_predictor(t, y_train_0, y_test_0, ntk_test_train)


In [ ]:
print(len(labels))
print(len(outputs))

29
29


In [ ]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.6395348837209303

In [ ]:
print(sum(torch.cat(labels).view(-1)))
print(sum(torch.cat(outputs).view(-1)))

tensor(71)
tensor(102)


In [ ]:
#@title Results

##CREDIT DATASET
model_name = 'LinearModel' 
batch_size = 32
num_epochs = 5
learning_rate = 0.001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 99.82619992275552 %
#roc_auc_score: 0.5

model_name = 'nlModel' 
batch_size = 32
num_epochs = 5
learning_rate = 0.001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 99.82619992275552 %
#0.5

model_name = 'dnnModel' 
batch_size = 32
num_epochs = 5
learning_rate = 0.001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 99.82619992275552 %
#0.5

##Recidivism
model_name = 'LinearModel' 
batch_size = 4
num_epochs = 10
learning_rate = 0.0001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 60.5668016194332 %
#roc_auc_score: 0.5717150592502894

model_name = 'nlModel' 
batch_size = 4
num_epochs = 10
learning_rate = 0.0001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 54.412955465587046 %
#roc_auc_score: 0.49925705794947994

model_name = 'dnnModel' 
batch_size = 4
num_epochs = 10
learning_rate = 0.0001
optimizer_name ='SGD'
input_size=train_dataset[0]["x"].shape[0]
output_size=2
#Accuracy of the network: 54.493927125506076 %
#roc_auc_score: 0.5